In [10]:
from keras.optimizers import RMSprop
import pandas as pd
import keras
import os
import seaborn as sns
import tensorflow
from keras import layers, regularizers, Input, Model
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder, MinMaxScaler, StandardScaler, OrdinalEncoder

In [11]:
"""
Global DF
"""
# list of root paths of all csv files in 'input/data' directory
csv_files = [f'dataset/{path}' for path in os.listdir('dataset') if path.endswith('.csv')]

# dataframe consisting of all csv files
df = pd.concat(pd.read_csv(_df, low_memory=False) for _df in csv_files)

In [12]:
"""
Clone of DF
"""
X = df.copy()
y = df['attack'].copy()
# id(X) == id(df), id(y) == id(df['attack'])

In [13]:
# classify columns by data type
continuous_attributes = [col for col in df.columns if df[col].dtype == 'float64']
discrete_attributes = [col for col in df.columns if str(df[col].dtype).startswith('int')]
categorical_attributes = [col for col in df.columns if df[col].dtype == 'object']

In [14]:
# TODO: Figure out how to preprocess features

# not sure whether to normalize or standardize continuous attributes

_scaler = StandardScaler()
# _scaler = MinMaxScaler()
for _col in continuous_attributes:
    """
    squash real valued features into the range [0, 1]
    """
    print(f'before standardizing/normalizing: {_col}', df[_col].describe(), '\n', sep='\n')
    X[_col] = _scaler.fit_transform(X[_col].values.reshape(-1, 1))
    print(f'after standardizing/normalizing: {_col}', df[_col].describe(), '\n', sep='\n')

# standardize discrete attributes
# _scaler = MinMaxScaler()
_scaler = StandardScaler()
for _col in discrete_attributes:
    print(f'before standardizing/normalizing: {_col}', df[_col].value_counts(), '\n', sep='\n')
    df[_col] = _scaler.fit_transform(df[_col].values.reshape(-1, 1))
    print(f'after standardizing/normalizing: {_col}', df[_col].value_counts(), '\n', sep='\n')

# enumerate categorical attributes
le = LabelEncoder()
for _col in categorical_attributes:
    print(f'before enumerating: {_col}', df[_col].value_counts(), '\n', sep='\n')
    try:
        df[_col] = le.fit_transform(df[_col])
    except TypeError:
        df[_col] = le.fit_transform(df[_col].astype(str))
    print(f'after enumerating: {_col}', df[_col].value_counts(), '\n',  sep='\n')

before standardizing/normalizing: stime
count    3.668522e+06
mean     1.528050e+09
std      2.581498e+05
min      1.526344e+09
25%      1.528085e+09
50%      1.528096e+09
75%      1.528099e+09
max      1.529381e+09
Name: stime, dtype: float64


after standardizing/normalizing: stime
count    3.668522e+06
mean     1.528050e+09
std      2.581498e+05
min      1.526344e+09
25%      1.528085e+09
50%      1.528096e+09
75%      1.528099e+09
max      1.529381e+09
Name: stime, dtype: float64


before standardizing/normalizing: ltime
count    3.668522e+06
mean     1.528050e+09
std      2.581507e+05
min      1.526344e+09
25%      1.528085e+09
50%      1.528096e+09
75%      1.528099e+09
max      1.529382e+09
Name: ltime, dtype: float64


after standardizing/normalizing: ltime
count    3.668522e+06
mean     1.528050e+09
std      2.581507e+05
min      1.526344e+09
25%      1.528085e+09
50%      1.528096e+09
75%      1.528099e+09
max      1.529382e+09
Name: ltime, dtype: float64


before standardizi

In [15]:
"""
Approach One: Failed
"""
# TODO: Set up the encoder to compute the reconstruction error
# This is the size of our encoded representations
encoding_dim = 30  # compression factor of ~2/3 per layer, assuming the input is 45 floats
model = Sequential()
# This is our input
model.add(Dense(46, input_shape=(46,), activation='relu'))
# "encoded" is the encoded representation of the input
model.add(Dense(encoding_dim, activation='relu'))
# "decoded" is the lossy reconstruction of the input
model.add(Dense(46, activation='softmax'))

model.compile(optimizer=keras.optimizers.Adadelta(), loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 46)                2162      
                                                                 
 dense_8 (Dense)             (None, 30)                1410      
                                                                 
 dense_9 (Dense)             (None, 46)                1426      
                                                                 
Total params: 4,998
Trainable params: 4,998
Non-trainable params: 0
_________________________________________________________________


In [16]:
"""
Approach Two: Failed
"""
# Nodes per layer
input_size = 46 # 45 features + 1 label
hidden_size = 32 # size of hidden layer
code_size = 16 # size of latent space

# Encode
_input = Input(shape=(input_size,))
hidden_1 = Dense(hidden_size, activation='relu')(_input)

# Latent Space
code = Dense(code_size, activation='relu')(hidden_1)

# Decode
hidden_2 = Dense(hidden_size, activation='relu')(code)
_output = Dense(input_size, activation='relu')(hidden_2)
autoencoder = Model(_input, _output)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 46)]              0         
                                                                 
 dense_10 (Dense)            (None, 32)                1504      
                                                                 
 dense_11 (Dense)            (None, 16)                528       
                                                                 
 dense_12 (Dense)            (None, 32)                544       
                                                                 
 dense_13 (Dense)            (None, 46)                1518      
                                                                 
Total params: 4,094
Trainable params: 4,094
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Split data into training and testing sets
# For autoencoder, input and output are the same
X_train, X_test, *labels = train_test_split(X, y, test_size=0.2, random_state=42)